In [13]:
import json

data = []

# Example for image
data.append({
  "messages": [
    {
      "role": "user",
      "content": [
        {"type": "image", "image": "Data\Images\earthquake\download.jpg"},
        {"type": "text", "text": "Describe this image in detail."}
      ]
    },
    {
      "role": "assistant",
      "content": "This image shows a distructed building due to earthquake since there is a lot of debris is setteled in this area"
    }
  ]
})

# Example for video
data.append({
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "video",
          "image": "Data\Images\earthquake\download2.jpg",
          "max_pixels": 360 * 420,
          "fps": 1.0
        },
        {"type": "text", "text": "What can be inferred from this surveillance video?"}
      ]
    },
    {
      "role": "assistant",
      "content": "This Image shows the damaged houses which might be made of wood, this might be due to earthquake"
    }
  ]
})

# Save to file
with open("train_dataset.json", "w") as f:
    json.dump(data, f, indent=2)


In [6]:
torch.cuda.empty_cache()

  Using cached transformers-4.38.2-py3-none-any.whl.metadata (130 kB)
  Using cached tokenizers-0.15.2-cp311-none-win_amd64.whl.metadata (6.8 kB)
Using cached transformers-4.38.2-py3-none-any.whl (8.5 MB)
   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   ------------------------------- -------- 786.4/991.5 kB 6.7 MB/s eta 0:00:01
   ---------------------------------------- 991.5/991.5 kB 6.7 MB/s eta 0:00:00
Using cached tokenizers-0.15.2-cp311-none-win_amd64.whl (2.2 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.0.dev0
    Uninstalling transformers-4.52.0.dev0:
      Successfully uninstalled transformers-4.52.0.dev0


  You can safely remove it manually.
